<a href="https://colab.research.google.com/github/SeohyunLyoo/Study/blob/main/Attention%20-%20Summary.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [74]:
import warnings
warnings.filterwarnings('ignore')

In [75]:
import tensorflow as tf
import nltk
import numpy as np
import pandas as pd
import re
import matplotlib.pyplot as plt
from nltk.corpus import stopwords
from bs4 import BeautifulSoup
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [76]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [77]:
%cd drive/MyDrive/Study



[Errno 2] No such file or directory: 'drive/MyDrive/Study'
/content/drive/MyDrive/Study


# **1. 데이터 준비 및 불용어 취합**

In [78]:
data = pd.read_csv('Reviews.csv', nrows = 100000)
print(len(data)), print(data.info())

100000
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 10 columns):
 #   Column                  Non-Null Count   Dtype 
---  ------                  --------------   ----- 
 0   Id                      100000 non-null  int64 
 1   ProductId               100000 non-null  object
 2   UserId                  100000 non-null  object
 3   ProfileName             99994 non-null   object
 4   HelpfulnessNumerator    100000 non-null  int64 
 5   HelpfulnessDenominator  100000 non-null  int64 
 6   Score                   100000 non-null  int64 
 7   Time                    100000 non-null  int64 
 8   Summary                 99998 non-null   object
 9   Text                    100000 non-null  object
dtypes: int64(5), object(5)
memory usage: 7.6+ MB
None


(None, None)

In [79]:
data.isnull().sum()

,0
Id,0
ProductId,0
UserId,0
ProfileName,6
HelpfulnessNumerator,0
HelpfulnessDenominator,0
Score,0
Time,0
Summary,2
Text,0


In [80]:
df = data[['Text', 'Summary']]
df.head()

,Text,Summary
0,I have bought several of the Vitality canned d...,Good Quality Dog Food
1,Product arrived labeled as Jumbo Salted Peanut...,Not as Advertised
2,This is a confection that has been around a fe...,"""Delight"" says it all"
3,If you are looking for the secret ingredient i...,Cough Medicine
4,Great taffy at a great price. There was a wid...,Great taffy


In [81]:
### Text 및 Summary 항목 중복 여부 체크
df['Text'].nunique(), df['Summary'].nunique()

(88426, 72348)

In [82]:
### Text열 중복 데이터 삭제 (Summary는 중복 데이터 유지, Text에 대한 동일 Summary 지원)
df.drop_duplicates(subset = ['Text'], inplace = True)
print(len(df))

88426


In [83]:
### Text 및 Summary Null 값 여부 확인
df['Text'].isnull().sum(), df['Summary'].isnull().sum()

(0, 1)

In [84]:
### Summary 항목 null 값인 데이터 삭제
df.dropna(axis=0, inplace=True)
df['Text'].isnull().sum(), df['Summary'].isnull().sum()

(0, 0)

In [85]:
### NLTK 불용어 취합
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))
len(stop_words)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


179

In [86]:
print(stop_words)

{'is', 'few', 'didn', "you've", 'they', 'we', "haven't", 'was', 'how', 'theirs', 'until', 'then', 'it', 'll', 'shan', 'too', 'on', 'had', 'yours', 'before', "shouldn't", 'out', 'why', 'don', 'over', 'all', 'just', 'once', 'ma', 'same', 'i', 'shouldn', 'here', 'each', 'ours', "shan't", "couldn't", 'after', "aren't", 'above', "isn't", 'very', 'having', 'when', 'only', 'won', 'been', 'in', "doesn't", 'has', 'ourselves', 'but', 'down', 'with', 'are', 'o', 'because', 'which', 'him', "don't", 'haven', 'hasn', "needn't", 'wasn', 'couldn', 'against', 'you', 'off', 'm', 'y', "you're", 'by', 'their', 'mightn', 'our', 'hers', 'from', 'his', 'myself', "it's", 'and', 'will', 'can', 'most', 't', 'my', 'again', 'through', 'if', 'those', 'wouldn', 'being', 'than', 'into', 'own', "hasn't", 'a', "should've", 'ain', "she's", 'the', "you'd", 'your', 'himself', 'them', 'should', 'not', "mustn't", 'these', 're', 'yourself', 'where', 'while', 'aren', 'hadn', 'other', 'am', 'needn', 'what', 'of', 've', 'under

# **2. 데이터 전처리**

In [87]:
!pip install contractions
import contractions

In [88]:
def preprocess_sentence(sentence, remove_stopwords=True):

    if not isinstance(sentence, str):
        sentence = str(sentence)

    sentence = sentence.lower()                             ### 텍스트 소문자화
    sentence = BeautifulSoup(sentence, "lxml").text         ### ,  등의 html 태그 제거
    sentence = re.sub(r'\([^)]*\)', '', sentence)           ### 괄호로 닫힌 문자열  제거 Ex) my husband (and myself) for => my husband for
    sentence = re.sub('"','', sentence)                     ### 쌍따옴표 " 제거

    ###sentence = ' '.join([contractions[t] if t in contractions else t for t in sentence.split(" ")])           ### 약어 정규화
    sentence = ' '.join([contractions.fix(t) for t in sentence.split(" ")])

    sentence = re.sub(r"'s\b","",sentence)                  ### 소유격 제거. Ex) roland's -> roland
    sentence = re.sub("[^a-zA-Z]", " ", sentence)           ### 영어 외 문자(숫자, 특수문자 등) 공백으로 변환
    sentence = re.sub('[m]{2,}', 'mm', sentence)            ### m이 3개 이상이면 2개로 변경. Ex) ummmmmmm yeah -> umm yeah

    ### 불용어 제거
    if remove_stopwords:
        tokens = ' '.join(word for word in sentence.split() if not word in stop_words if len(word) > 1)
    ### 불용어 미제거 (Summary)
    else:
        tokens = ' '.join(word for word in sentence.split() if len(word) > 1)

    return tokens

In [89]:
### Text : 불용어 제거, Summary : 불용어 유지
temp_text = 'Everything I bought was great, infact I ordered twice and the third ordered was<br />for my mother and father.'
temp_summary = 'Great way to start (or finish) the day!!!'
print(preprocess_sentence(temp_text))
print(preprocess_sentence(temp_summary, 0))

everything bought great infact ordered twice third ordered wasfor mother father
great way to start the day


In [90]:
### Text 전처리
clean_text = []

for s in df['Text']:
    clean_text.append(preprocess_sentence(s))

print(clean_text[:5])

['bought several vitality canned dog food products found good quality product looks like stew processed meat smells better labrador finicky appreciates product better', 'product arrived labeled jumbo salted peanuts peanuts actually small sized unsalted sure error vendor intended represent product jumbo', 'confection around centuries light pillowy citrus gelatin nuts case filberts cut tiny squares liberally coated powdered sugar tiny mouthful heaven chewy flavorful highly recommend yummy treat familiar story lewis lion witch wardrobe treat seduces edmund selling brother sisters witch', 'looking secret ingredient robitussin believe found got addition root beer extract ordered made cherry soda flavor medicinal', 'great taffy great price wide assortment yummy taffy delivery quick taffy lover deal']


In [91]:
### Summary 전처리
clean_summary = []

for s in df['Summary']:
    clean_summary.append(preprocess_sentence(s, 0))

clean_summary[:5]

['good quality dog food',
 'not as advertised',
 'delight says it all',
 'cough medicine',
 'great taffy']

In [92]:
df['Text'] = clean_text
df['Summary'] = clean_summary
df.head()

,Text,Summary
0,bought several vitality canned dog food produc...,good quality dog food
1,product arrived labeled jumbo salted peanuts p...,not as advertised
2,confection around centuries light pillowy citr...,delight says it all
3,looking secret ingredient robitussin believe f...,cough medicine
4,great taffy great price wide assortment yummy ...,great taffy


In [93]:
# 길이가 공백인 샘플은 NULL 값으로 변환
df.replace('', np.nan, inplace=True)
print(df.isnull().sum())

Text        0
Summary    56
dtype: int64


In [94]:
### SUMMARY 항목이 NULL인 경우 해당 건 삭제
df.dropna(axis=0, inplace=True)
len(df)

88369

In [95]:
df.isnull().sum()

,0
Text,0
Summary,0


In [96]:
### Text / Summary Padding Size 결정
text_max_len = 50
summary_max_len = 10

In [97]:
df = df[df['Text'].apply(lambda x: len(x.split()) <= text_max_len)]
df = df[df['Summary'].apply(lambda x: len(x.split()) <= summary_max_len)]
print('전체 샘플수 :',(len(df)))

전체 샘플수 : 67610


In [98]:
df[:5]

,Text,Summary
0,bought several vitality canned dog food produc...,good quality dog food
1,product arrived labeled jumbo salted peanuts p...,not as advertised
2,confection around centuries light pillowy citr...,delight says it all
3,looking secret ingredient robitussin believe f...,cough medicine
4,great taffy great price wide assortment yummy ...,great taffy


In [99]:
### decoder_input, decoder_output 시작/종료 토큰 추가
df['decoder_input'] = df['Summary'].apply(lambda x : 'sostoken ' + x)
df['decoder_output'] = df['Summary'].apply(lambda x : x + ' eostoken')
df.head()

,Text,Summary,decoder_input,decoder_output
0,bought several vitality canned dog food produc...,good quality dog food,sostoken good quality dog food,good quality dog food eostoken
1,product arrived labeled jumbo salted peanuts p...,not as advertised,sostoken not as advertised,not as advertised eostoken
2,confection around centuries light pillowy citr...,delight says it all,sostoken delight says it all,delight says it all eostoken
3,looking secret ingredient robitussin believe f...,cough medicine,sostoken cough medicine,cough medicine eostoken
4,great taffy great price wide assortment yummy ...,great taffy,sostoken great taffy,great taffy eostoken


In [100]:
### Encoder / Decoder의 레이블 지정
encoder_input  = np.array(df['Text'])
decoder_input  = np.array(df['decoder_input'])
decoder_target = np.array((df['decoder_output']))

In [101]:
### 훈련/테스트 데이터 분리
indices = np.arange(encoder_input.shape[0])
np.random.shuffle(indices)

In [102]:
encoder_input = encoder_input[indices]
decoder_input = decoder_input[indices]
decoder_target = decoder_target[indices]

In [103]:
n_of_val = int(len(encoder_input)*0.2)
print('테스트 데이터의 수 :',n_of_val)

테스트 데이터의 수 : 13522


In [104]:
encoder_input_train = encoder_input[:-n_of_val]
decoder_input_train = decoder_input[:-n_of_val]
decoder_target_train = decoder_target[:-n_of_val]

encoder_input_test = encoder_input[-n_of_val:]
decoder_input_test = decoder_input[-n_of_val:]
decoder_target_test = decoder_target[-n_of_val:]

In [105]:
print('훈련 데이터의 개수 :', len(encoder_input_train))
print('훈련 레이블의 개수 :',len(decoder_input_train))
print('테스트 데이터의 개수 :',len(encoder_input_test))
print('테스트 레이블의 개수 :',len(decoder_input_test))

훈련 데이터의 개수 : 54088
훈련 레이블의 개수 : 54088
테스트 데이터의 개수 : 13522
테스트 레이블의 개수 : 13522


# **3. 정수 인코딩**

**1) 단어 집합 생성**

In [106]:
src_tokenizer = Tokenizer()
src_tokenizer.fit_on_texts(encoder_input_train)

In [107]:
### 단어 등장 횟수 7회 미만 단어들은 자연어 처리에서 제외
threshold = 7

total_cnt = len(src_tokenizer.word_index)   ### 단어의 수
rare_cnt = 0                                ### 등장 빈도수가 threshold보다 작은 단어의 개수를 카운트
total_freq = 0                              ### 훈련 데이터의 전체 단어 빈도수 총 합
rare_freq = 0                               ### 등장 빈도수가 threshold보다 작은 단어의 등장 빈도수의 총 합

### 단어와 빈도수의 쌍(pair)을 key와 value로 받는다.
for key, value in src_tokenizer.word_counts.items():
    total_freq = total_freq + value

    ### 단어의 등장 빈도수가 threshold보다 작으면
    if(value < threshold):
        rare_cnt = rare_cnt + 1
        rare_freq = rare_freq + value

print('단어 집합(vocabulary)의 크기 :',total_cnt)
print('등장 빈도가 %s번 이하인 희귀 단어의 수: %s'%(threshold - 1, rare_cnt))
print('단어 집합에서 희귀 단어를 제외시킬 경우의 단어 집합의 크기 %s'%(total_cnt - rare_cnt))
print("단어 집합에서 희귀 단어의 비율:", (rare_cnt / total_cnt)*100)
print("전체 등장 빈도에서 희귀 단어 등장 빈도 비율:", (rare_freq / total_freq)*100)

단어 집합(vocabulary)의 크기 : 32362
등장 빈도가 6번 이하인 희귀 단어의 수: 24012
단어 집합에서 희귀 단어를 제외시킬 경우의 단어 집합의 크기 8350
단어 집합에서 희귀 단어의 비율: 74.1981336134973
전체 등장 빈도에서 희귀 단어 등장 빈도 비율: 3.317728164477665


In [108]:
encoder_input_train[:3]

array(['love fennel seeds perfect combination spicy sweet besides help digestion really good thing handy instead sweets',
       'really sure really power bones dog loves taste complain buy lots zuke products well golden loves',
       'hard find crunchy items gluten free world hits spot melt summer heat vegas thank'],
      dtype=object)

In [109]:
### Encoder 단어 사전의 크기를 8,000으로 제한
src_vocab = 8000

src_tokenizer = Tokenizer(num_words = src_vocab)
src_tokenizer.fit_on_texts(encoder_input_train)

### 텍스트 시퀀스를 정수 시퀀스로 변경
encoder_input_train = src_tokenizer.texts_to_sequences(encoder_input_train)
encoder_input_test  = src_tokenizer.texts_to_sequences(encoder_input_test)

encoder_input_train[:3]

[[6,
  5135,
  443,
  59,
  635,
  281,
  38,
  1632,
  351,
  1815,
  12,
  2,
  160,
  1159,
  277,
  2068],
 [12, 118, 12, 1633, 735, 32, 46, 4, 1832, 17, 473, 2144, 127, 27, 1286, 46],
 [97, 23, 328, 526, 159, 52, 663, 1853, 1430, 1119, 692, 531, 5731, 295]]

In [110]:
tar_tokenizer = Tokenizer()
tar_tokenizer.fit_on_texts(decoder_input_train)

In [111]:
### Decoder 단어 사전의 크기를 8,000으로 제한
tar_vocab = 2000
tar_tokenizer = Tokenizer(num_words = tar_vocab)
tar_tokenizer.fit_on_texts(decoder_input_train)
tar_tokenizer.fit_on_texts(decoder_target_train)

### 텍스트 시퀀스를 정수 시퀀스로 변환
decoder_input_train  = tar_tokenizer.texts_to_sequences(decoder_input_train)
decoder_target_train = tar_tokenizer.texts_to_sequences(decoder_target_train)
decoder_input_test   = tar_tokenizer.texts_to_sequences(decoder_input_test)
decoder_target_test  = tar_tokenizer.texts_to_sequences(decoder_target_test)

**2) 빈 샘플 제거**